In [1]:
import cv2
import numpy as np
import csv

# Function to overlay image with transparency
def overlay_image_alpha(background, overlay, position, alpha_mask):
    x, y = position
    h, w = overlay.shape[0], overlay.shape[1]

    # Ensure overlay and background have the same shape
    overlay = cv2.resize(overlay, (w, h))

    # Resize alpha mask to match overlay dimensions
    alpha_mask_resized = cv2.resize(alpha_mask, (w, h))

    # Apply the alpha mask to the overlay
    alpha = alpha_mask_resized / 255.0
    blended = (alpha * overlay + (1 - alpha) * background[y:y+h, x:x+w]).astype(np.uint8)

    # Replace the region of interest with the blended image
    background[y:y+h, x:x+w] = blended

# Function to overlay sunglasses on Jaime Lannister's image
def overlay_sunglasses_jaime_lannister(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Load eye cascade classifier
    eye_cascade_path = 'C:\\Users\\abhin\\Downloads\\Train\\third-party\\frontalEyes35x16.xml'
    eye_cascade = cv2.CascadeClassifier(eye_cascade_path)
    
    # Detect faces using the eye cascade
    faces = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    
    # Check if at least two eyes are detected
    if len(faces) < 2:
        print(f"Less than two eyes detected: {len(faces)}")
        return None
    
    # Display the number of faces detected
    print(f"Number of faces detected: {len(faces)}")
    
    # Iterate over detected faces
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = image[y:y+h, x:x+w]

        # Detect eyes within the face ROI
        eyes = eye_cascade.detectMultiScale(roi_gray)
        
        # Ensure at least two eyes are detected
        if len(eyes) >= 2:
            # Sort the detected eyes by x-coordinate
            eyes = sorted(eyes, key=lambda e: e[0])

            # Get the coordinates of the two eyes
            eye1, eye2 = eyes[0], eyes[1]

            # Calculate the center of the eyes
            eye_center = ((eye1[0] + eye2[0] + eye2[2]) // 2, (eye1[1] + eye2[1] + eye2[3]) // 2)
            
            # Calculate the width of the sunglasses based on the eyes
            sunglasses_width = eye2[0] + eye2[2] - eye1[0]
            scale_factor = sunglasses_width / glasses.shape[1]

            # Resize the sunglasses image
            new_sunglasses = cv2.resize(glasses, (0, 0), fx=scale_factor, fy=scale_factor)

            # Calculate the position to place the sunglasses
            x_offset = eye_center[0] - new_sunglasses.shape[1] // 2
            y_offset = eye_center[1] - new_sunglasses.shape[0] // 2

            # Overlay the sunglasses on the face
            overlay_image_alpha(roi_color, new_sunglasses[:, :, 0:3], (x_offset, y_offset), new_sunglasses[:, :, 3])
        else:
            print("Less than two eyes detected in the ROI.")

    return image

# Function to overlay mustache on Jaime Lannister's image
def overlay_mustache_jaime_lannister(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Load nose cascade classifier
    nose_cascade_path = 'C:\\Users\\abhin\\Downloads\\Train\\third-party\\Nose18x15.xml'
    nose_cascade = cv2.CascadeClassifier(nose_cascade_path)
    
    # Detect faces using the nose cascade
    faces = nose_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    
    # Check if at least one nose is detected
    if len(faces) == 0:
        print("Nose not detected.")
        return None
    
    # Display the number of faces detected
    print(f"Number of faces detected: {len(faces)}")
    
    # Iterate over detected faces
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = image[y:y+h, x:x+w]

        # Detect nose within the face ROI
        noses = nose_cascade.detectMultiScale(roi_gray)

        # Ensure at least one nose is detected
        if len(noses) > 0:
            # Assuming only one nose is detected for simplicity
            nx, ny, nw, nh = noses[0]

            # Calculate the position to place the mustache
            mustache_width = nw
            scale_factor = mustache_width / mustache.shape[1]
            new_mustache = cv2.resize(mustache, (0, 0), fx=scale_factor, fy=scale_factor)

            x_offset_mustache = nx + nw // 2 - new_mustache.shape[1] // 2
            y_offset_mustache = ny + nh + 10  # Adjust this offset based on your image and positioning

            # Overlay the mustache on the face
            overlay_image_alpha(roi_color, new_mustache[:, :, 0:3], (x_offset_mustache, y_offset_mustache), new_mustache[:, :, 3])
        else:
            print("Nose not detected.")

    return image

# Function to save BGR values to CSV
def save_bgr_to_csv(image, csv_path):
    bgr_flat = image.reshape(-1, 3)
    
    try:
        with open(csv_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(bgr_flat)
        print(f"Saved data to {csv_path}")
    except Exception as e:
        print(f"Error saving CSV: {e}")

# Load images and haarcascade files
jaime_before_path = "C:\\Users\\abhin\\Downloads\\Train\\Jamie_Before.jpg"
jaime_after_path = "C:\\Users\\abhin\\Downloads\\Train\\Jamie_After.jpg"
mustache_path = "C:\\Users\\abhin\\Downloads\\Train\\mustache.png"
glasses_path = "C:\\Users\\abhin\\Downloads\\Train\\glasses.png"

# Load images
jaime_before = cv2.imread(jaime_before_path)
jaime_after = cv2.imread(jaime_after_path)
mustache = cv2.imread(mustache_path, cv2.IMREAD_UNCHANGED)
glasses = cv2.imread(glasses_path, cv2.IMREAD_UNCHANGED)

# Overlay sunglasses on Jaime Lannister's image
jaime_sunglasses = overlay_sunglasses_jaime_lannister(jaime_before.copy())

if jaime_sunglasses is not None:
    # Display the image with sunglasses
    cv2.imshow("Jaime Lannister with Sunglasses", jaime_sunglasses)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Save Jaime Lannister's image with sunglasses to CSV
    jaime_sunglasses_csv_path = "jaime_sunglasses.csv"
    save_bgr_to_csv(jaime_sunglasses, jaime_sunglasses_csv_path)

# Overlay mustache on Jaime Lannister's image
jaime_mustache = overlay_mustache_jaime_lannister(jaime_after.copy())

if jaime_mustache is not None:
    # Display the image with mustache
    cv2.imshow("Jaime Lannister with Mustache", jaime_mustache)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Save Jaime Lannister's image with mustache to CSV
    jaime_mustache_csv_path = "jaime_mustache.csv"
    save_bgr_to_csv(jaime_mustache, jaime_mustache_csv_path)


Less than two eyes detected: 1
Number of faces detected: 2


ValueError: operands could not be broadcast together with shapes (20,40) (20,40,3) 

In [ ]:
import os

# Print current working directory
print("Current working directory:", os.getcwd())

# Specify path for saving CSV
jaime_sunglasses_csv_path = "jaime_sunglasses.csv"

# Save Jaime Lannister's image with sunglasses to CSV
try:
    save_image_to_csv(jaime_sunglasses, jaime_sunglasses_csv_path)
    print(f"Saved Jaime Lannister with sunglasses to {jaime_sunglasses_csv_path}")
except Exception as e:
    print(f"Error saving file: {e}")
